# 클러스터 구성 과정
# 1. node 0개로 클러스터 생성 (secondary cidr 설정을 위해)
# 2. bootstrap (secondary cidr, aws load balancer controller)
# 3. 노드 갯수 원하는 양으로 조절
## 의 과정을 거쳐야 한다. (aws는 k8s에 최적화가 덜되서...)

Deprecated (이전 절차)
절차

1. node_size.yml => 0
2. terraform apply => output으로 eniconfig.yaml 파일 작성
3. aws eks --region us-east-1 update-kubeconfig --name test-eks-cluster
4. kubectl create -f ./k8s-manifests/eni-configs/.
5. kubectl set env daemonset aws-node -n kube-system AWS_VPC_K8S_CNI_CUSTOM_NETWORK_CFG=true
6. kubectl set env daemonset aws-node -n kube-system ENI_CONFIG_LABEL_DEF=failure-domain.beta.kubernetes.io/zone
7. node_size.yml => 0
8. terraform apply
9. node_size.yml => cluster size you want
10. terraform apply

절차

1. node_size.yml => 0
2. terraform apply => output으로 eniconfig.yaml 파일 작성
3. boot strap 과정에 개편을 거쳐서 이젠 boot strap 데이터를 terraform 으로 전송해서, null_resource를 통해 실행시킨다.
4. node_size.yml => cluster size you want
5. terraform apply --auto-approve
6. node_size.yml => 원하는 사이즈
7. terraform apply --auto-approve

여기다가 초기 정보 (cluster name, region) 등 몰아서 설정하게 해야 되는데...

In [1]:
#!terraform destroy --auto-approve

# 1. 초기 클러스터 크기

In [2]:
import yaml
def change_node_size_yml(desired_size, max_size, min_size):
    with open('node_size.yml') as f:
        node_size = yaml.load(f, Loader=yaml.FullLoader)
        print('current_state',node_size)
        
    node_size['desired_size'] = desired_size
    node_size['min_size'] = min_size
    node_size['max_size'] = max_size
    
    print('after change', node_size)
    
    with open('node_size.yml', 'w') as f:
        yaml.dump(node_size,f)

In [3]:
desired_size = 0
max_size = 1 #maxsize는 0이 될 수 없다.
min_size = 0

In [4]:
change_node_size_yml(desired_size, max_size, min_size)

current_state {'desired_size': 1, 'max_size': 1, 'min_size': 1}
after change {'desired_size': 0, 'max_size': 1, 'min_size': 0}


# 2. Terraform 실행

#테라폼 설치 법 2023-06-25 현재 최신버전  1.5.1    
https://askubuntu.com/questions/983351/how-to-install-terraform-in-ubuntu

In [5]:
#mv terraform /usr/local/bin #컨테이너 안에서 실행했음

In [6]:
#!terraform --version

In [7]:
#!terraform init

In [8]:
#!terraform plan

In [17]:
#!terraform apply --auto-approve

In [10]:
#테라폼 버그? 인지 secondary cidr subnet이 한번에 생성이 안됨
#secondary cidr, subnet에 depends on을 붙이고 나서부터 한번에 만들어짐
#!terraform apply --auto-approve

# 3. bastion server에서 bootstrap 실행
# => 테라폼 null_resource로 대체

In [11]:
#혹시 bootstrap 이후 필요하면 sleep
#import time

#time.sleep(120)

# 4 node size -> 다시 늘리기

In [12]:
#node_size.yml => cluster size you want

In [13]:
#노드 그룹당 1개 , 노드그룹은 terraform에 2개로 설정되어 있음
#리소스 때문에 kubeflow 설치 못하는거 같아서 2배로 늘려봄
desired_size = 1
max_size = 1
min_size = 1

change_node_size_yml(desired_size, max_size, min_size)

current_state {'desired_size': 0, 'max_size': 1, 'min_size': 0}
after change {'desired_size': 1, 'max_size': 1, 'min_size': 1}


In [18]:
#!terraform apply --auto-approve

In [15]:
#잠깐 쉴때는 노드 사이즈만 줄여도 괜찮네
#!terraform apply --auto-approve

In [16]:
#!terraform destroy --auto-approve